# 作業 : (Kaggle)鐵達尼生存預測
https://www.kaggle.com/c/titanic

# [作業目標]
- 試著模仿範例寫法, 在鐵達尼生存預測中, 觀察填補缺值以及 標準化 / 最小最大化 對數值的影響

# [作業重點]
- 觀察替換不同補缺方式, 對於特徵的影響 (In[4]~In[6], Out[4]~Out[6])
- 觀察替換不同特徵縮放方式, 對於特徵的影響 (In[7]~In[8], Out[7]~Out[8])

In [65]:
# 做完特徵工程前的所有準備 (與前範例相同)
import pandas as pd
import numpy as np
import copy
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
import warnings
warnings.filterwarnings("ignore")

data_path = 'data/'
df_train = pd.read_csv(data_path + 'titanic_train.csv')
df_test = pd.read_csv(data_path + 'titanic_test.csv')

train_Y = df_train['Survived']
ids = df_test['PassengerId']
df_train = df_train.drop(['PassengerId', 'Survived'] , axis=1)
df_test = df_test.drop(['PassengerId'] , axis=1)
df = pd.concat([df_train,df_test])
df.head()


,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [67]:
#只取 int64, float64 兩種數值型欄位, 存於 num_features 中
num_features = []
for dtype, feature in zip(df.dtypes, df.columns):
    if dtype == 'float64' or dtype == 'int64':
        num_features.append(feature)
print(f'{len(num_features)} Numeric Features : {num_features}\n')


5 Numeric Features : ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare']



In [87]:
# 削減文字型欄位, 只剩數值型欄位
df = df[num_features]
train_num = train_Y.shape[0]
df.head()

0      0
1      1
2      1
3      1
4      0
5      0
6      0
7      0
8      1
9      1
10     1
11     1
12     0
13     0
14     0
15     1
16     0
17     1
18     0
19     1
20     0
21     1
22     1
23     1
24     0
25     1
26     0
27     0
28     1
29     0
      ..
861    0
862    1
863    0
864    0
865    1
866    1
867    0
868    0
869    1
870    0
871    1
872    0
873    0
874    1
875    1
876    0
877    0
878    0
879    1
880    1
881    0
882    0
883    0
884    0
885    0
886    0
887    1
888    0
889    1
890    0
Name: Survived, Length: 891, dtype: int64


# 作業1
* 試著在補空值區塊, 替換並執行兩種以上填補的缺值, 看看何者比較好?

In [96]:
# 空值補 -1, 做羅吉斯迴歸
df_m1 = df.fillna(-1)
df_m2 = df.fillna(0)
df_m3 = df.fillna(100)
#print(df_m1.shape)
train_X = df_m1[:train_num]
train_X2 = df_m2[:train_num]
train_X3 = df_m3[:train_num]
#print(df_m1)
estimator = LogisticRegression()
print('fill with "-1:"',cross_val_score(estimator, train_X, train_Y, cv=5).mean())
print('fill with "0":',cross_val_score(estimator, train_X2, train_Y, cv=5).mean())
print('fill with "100":',cross_val_score(estimator, train_X3, train_Y, cv=5).mean())
#print(cross_val_score(estimator, train_X, train_Y, cv=5))
#分別填入100,0，結果差異不大

fill with "-1:" 0.6960299128976762
fill with "0": 0.6971535084032942
fill with "100": 0.6870095507213844


# 作業2
* 使用不同的標準化方式 ( 原值 / 最小最大化 / 標準化 )，搭配羅吉斯迴歸模型，何者效果最好?

In [107]:
"""
Yur Code Here
"""
df_m4 = df.fillna(df.mean())                   #填入平均值
print(df.mean())
df_m5 = df.fillna(0)
df_m5 = MinMaxScaler().fit_transform(df_m5)   #填入0搭配最小最大化
df_m6 = df.fillna(0)
df_m6 = StandardScaler().fit_transform(df_m6) #填入0搭配標準化
#print(df_m1.shape)
train_X4 = df_m4[:train_num]
train_X5 = df_m5[:train_num]
train_X6 = df_m6[:train_num]
train_X7 = df_m7[:train_num]
#print(df_m1)
estimator = LogisticRegression()
print('fill with "填入平均值":',cross_val_score(estimator, train_X4, train_Y, cv=5).mean())
print('fill with "填入0搭配最小最大化":',cross_val_score(estimator, train_X5, train_Y, cv=5).mean())
print('fill with "填入0搭配標準化":',cross_val_score(estimator, train_X6, train_Y, cv=5).mean())


Pclass     2.294882
Age       29.881138
SibSp      0.498854
Parch      0.385027
Fare      33.295479
dtype: float64
fill with "填入平均值": 0.6981761033723469
fill with "填入0搭配最小最大化": 0.7016289882888318
fill with "填入0搭配標準化": 0.6982582017719778
